## LangChain Expression Language(LCEL)

https://python.langchain.com/v0.1/docs/expression_language/why/

### 기본 구조: 프롬프트 + 모델 + 출력 파서


In [2]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [8]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [9]:
#API KEY 저장을 위한 os 라이브러리 호출
import os

os.environ['LANGCHAIN_PROJECT'] = 'LCEL'
print(f"[LANGCHAIN_PROJECT]\n{os.environ['LANGCHAIN_PROJECT']}")

[LANGCHAIN_PROJECT]
LCEL


In [6]:
!pip install -qU langchain-teddynote


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [10]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LCEL")

LangSmith 추적을 시작합니다.
[프로젝트명]
LCEL


### 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿
  - `template`: 템플릿 문자열. 문자열 내에서 중괄호 `{}`는 변수를 나타냄
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의함

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트

In [11]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [12]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [13]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [14]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [16]:
!pip install langchain langchain-openai

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [17]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=2048,
    temperature=0.1,
)

### Chain 생성

#### LCEL(LangChain Expression Language)


```
chain = prompt | model | output_parser
```

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달


In [18]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI()

chain = prompt | model
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 쉽게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f2179867220>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f2179875360>, root_client=<openai.OpenAI object at 0x7f2179865300>, root_async_client=<openai.AsyncOpenAI object at 0x7f2179867280>, model_kwargs={}, openai_api_key=SecretStr('**********'))

### invoke() 호출

- python 딕셔너리 형태(키: 값)로 입력값을 전달
- invoke() 함수 호출 시, 입력값을 전달

In [19]:
# input 딕셔너리에 주제 설정
input = {"topic": "인공지능의 학습 방법"}

In [20]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달
# 이를 통해 AI 모델이 생성한 메시지를 반환
chain.invoke(input)

AIMessage(content='인공지능의 학습 방법은 크게 지도학습, 비지도학습, 강화학습으로 나뉩니다.\n\n1. 지도학습: 레이블링된 데이터를 바탕으로 모델을 학습시키는 방법입니다. 입력 데이터와 정답 데이터를 모델에 제공하여 모델이 정답을 예측하도록 학습시킵니다. 예를 들어, 고양이 이미지와 해당 이미지가 고양이임을 나타내는 레이블을 제공하여 모델이 고양이를 인식하도록 학습시키는 것이 지도학습입니다.\n\n2. 비지도학습: 레이블링된 데이터 없이 입력 데이터만을 이용하여 모델을 학습시키는 방법입니다. 데이터 간의 패턴이나 관계를 발견하거나 데이터를 그룹화하는 데 사용됩니다. 예를 들어, 주어진 고객 데이터를 이용하여 유사한 특징을 가진 고객들을 그룹화하는 것이 비지도학습입니다.\n\n3. 강화학습: 보상 시스템을 통해 환경과 상호작용하며, 최적의 행동을 학습하는 방법입니다. 모델은 환경에서 어떤 행동을 취할 때 어떤 보상을 받는지 학습하고, 최적의 행동을 찾아가는 과정을 거칩니다. 예를 들어, 게임이나 로봇 제어 등에서 사용됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 445, 'prompt_tokens': 30, 'total_tokens': 475, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-98d3d617-32df-46f7-8690-d7ec2418a4f6-0', usage_metadata={'inpu

In [21]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능은 데이터를 입력받아 패턴을 학습하여 문제를 해결하는 능력을 갖고 있습니다. 이를 위해 인공지능은 학습 데이터를 사용하여 모델을 훈련시키고, 이를 통해 입력 데이터를 분석하고 판단할 수 있는 능력을 갖춥니다. 학습 방법에는 지도학습, 비지도학습, 강화학습 등이 있으며, 각각의 방법은 다양한 방식으로 데이터를 활용하여 모델을 향상시키는 목적을 가지고 있습니다. 이러한 학습 방법을 통해 인공지능은 점차적으로 더 나은 성능과 능력을 갖추게 됩니다.

### 출력파서(Output Parser)


In [22]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [23]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 쉽게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f2179867220>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f2179875360>, root_client=<openai.OpenAI object at 0x7f2179865300>, root_async_client=<openai.AsyncOpenAI object at 0x7f2179867280>, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [24]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능의 학습 원리"}
chain.invoke(input)

'인공지능의 학습 원리는 크게 지도학습, 비지도학습, 강화학습으로 나눌 수 있습니다.\n\n1. 지도학습: 인공지능 시스템에게 입력 데이터와 해당하는 정답을 함께 제공하여 학습시키는 방법입니다. 이때 시스템은 입력 데이터와 정답 사이의 패턴을 학습하여 새로운 데이터에 대한 예측을 할 수 있게 됩니다.\n\n2. 비지도학습: 정답이 주어지지 않은 입력 데이터만을 이용하여 시스템이 스스로 데이터 간의 패턴이나 관계를 찾아내는 방법입니다. 이를 통해 데이터의 특성이나 구조를 파악하거나 군집화, 이상치 탐지 등의 작업을 수행할 수 있습니다.\n\n3. 강화학습: 시스템이 환경과 상호작용하며 보상을 최대화하는 방향으로 학습하는 방법입니다. 시스템은 특정 행동을 취했을 때 얻는 보상을 통해 최적의 행동을 찾아나가는 과정을 거치게 됩니다.\n\n이러한 학습 원리를 통해 인공지능 시스템은 데이터를 분석하고 판단하는 능력을 향상시킬 수 있습니다.'

In [25]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능의 학습 원리는 데이터를 입력받아 패턴을 학습하고, 그 패턴을 기반으로 새로운 데이터를 예측하거나 의사결정을 내리는 과정입니다. 이를 위해 인공지능 시스템은 입력 데이터를 처리하고 분석하여 특징을 추출하고, 이를 바탕으로 모델을 학습시킵니다. 모델은 학습 데이터를 토대로 가중치를 조정하고 최적의 예측을 할 수 있도록 학습됩니다. 이렇게 학습된 모델은 새로운 데이터를 입력받아 예측하거나 분류하는 역할을 수행하게 됩니다. 이러한 방식으로 인공지능은 데이터를 기반으로 스스로 학습하고 결정을 내릴 수 있습니다.

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 가능
- `model_name` 역시 변경하여 테스트가 가능

In [26]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

In [27]:
# 체인을 구성
chain = prompt | model | output_parser

In [28]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
  - Waiter: Hi there! Welcome to our restaurant. How can I help you today?
  - You: Hi! I’d like to see the menu, please.
  - Waiter: Of course! Here you go. Can I get you anything to drink while you look?
  - You: Yes, I’d like a glass of water, please.
  - Waiter: Sure! Are you ready to order, or do you need more time?
  - You: I think I’m ready. I’d like to order the grilled chicken with a side salad.
  - Waiter: Great choice! Would you like any dressing for your salad?
  - You: Yes, please. I’ll have the vinaigrette.
  - Waiter: Perfect! I’ll get that for you right away.

- 한글 해석:
  - 웨이터: 안녕하세요! 저희 식당에 오신 것을 환영합니다. 어떻게 도와드릴까요?
  - 당신: 안녕하세요! 메뉴를 볼 수 있을까요?
  - 웨이터: 물론이죠! 여기 있습니다. 보시는 동안 음료를 드릴까요?
  - 당신: 네, 물 한 잔 주세요.
  - 웨이터: 알겠습니다! 주문할 준비가 되셨나요, 아니면 좀 더 시간이 필요하신가요?
  - 당신: 이제 준비가 된 것 같아요. 그릴에 구운 치킨과 사이드 샐러드를 주문할게요.
  - 웨이터: 좋은 선택입니다! 샐러드에 드레싱은 어떤 걸 원하시나요?
  - 당신: 네, 비네그레트 드레싱으로 주세요.
  - 웨이터: 완벽합니다! 바로 가져다 드리겠습니다.


In [29]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
  - Waiter: Good evening! How many people are in your party?
  - You: Good evening! It's just me.
  - Waiter: Great! Here’s the menu. Can I get you something to drink while you decide?
  - You: Yes, I’d like a glass of water, please.
  - Waiter: Of course! Are you ready to order, or do you need a few more minutes?
  - You: I’m ready. I would like the grilled chicken salad, please.
  - Waiter: Excellent choice! Would you like any dressing with that?
  - You: Yes, please. Can I have balsamic vinaigrette?
  - Waiter: Absolutely! I’ll be right back with your order.

- 한글 해석:
  - 웨이터: 좋은 저녁입니다! 몇 분이신가요?
  - 당신: 좋은 저녁입니다! 저 혼자입니다.
  - 웨이터: 좋습니다! 여기에 메뉴가 있습니다. 주문을 결정하는 동안 음료를 드릴까요?
  - 당신: 네, 물 한 잔 주세요.
  - 웨이터: 물론입니다! 주문할 준비가 되셨나요, 아니면 몇 분 더 필요하신가요?
  - 당신: 준비됐습니다. 그릴에 구운 치킨 샐러드로 주세요.
  - 웨이터: 훌륭한 선택입니다! 드레싱은 필요하신가요?
  - 당신: 네, 발사믹 비네그레트를 주세요.
  - 웨이터: 알겠습니다! 주문하신 대로 곧 가져다드리겠습니다.

In [30]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
A: Hi there! I’d like to order a pizza, please.
B: Sure! What size would you like?
A: I’ll have a large, please. 
B: Great! What toppings do you want?
A: I’d like pepperoni and mushrooms.
B: Sounds good! Would you like anything to drink?
A: Yes, I’d like a Coke, please.
B: Perfect! Your total comes to $25. How would you like to pay?
A: I’ll pay with my credit card.
B: Thank you! Your pizza will be ready in about 20 minutes.

- 한글 해석:
A: 안녕하세요! 피자를 주문하고 싶어요.
B: 물론이죠! 어떤 사이즈로 하시겠어요?
A: 큰 사이즈로 해주세요.
B: 좋습니다! 어떤 토핑을 원하시나요?
A: 페퍼로니와 버섯을 추가해주세요.
B: 좋네요! 음료수는 필요하신가요?
A: 네, 콜라 하나 주세요.
B: 완벽해요! 총 금액은 25달러입니다. 어떻게 결제하시겠어요?
A: 신용카드로 결제할게요.
B: 감사합니다! 피자는 약 20분 후에 준비될 거예요.

### [실습] 템플릿을 변경하여 나만의 여행 가이드 챗봇 만들기

- 위의 프롬프트를 아래 주제에 맞게 **변경** 해보기
1. 페르소나: 10년차 여행 가이드
2. 3일간 가성비 여행 계획을 세워주는 챗봇 생성
3. `{question}` 에는 여행갈 나라와 도시를 사용자에게 입력받음
4. `answer` 변수를 출력하여 챗봇의 답변 결과 확인 
5. `Langsmith` 에 접속하여 실행 내용 확인

In [44]:
tour_guide_template = """
당신은 10년차 여행 가이드입니다. 주어진 지역에서 3일간 가성비 좋은 여행 계획을 세워주세요.

# 지역
{location}
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성
prompt = PromptTemplate.from_template(tour_guide_template)

# ChatOpenAI 챗모델을 초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

# 체인을 구성
chain = prompt | model | output_parser

In [46]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"location": "브리스톨"})
# 스트리밍 출력
stream_response(answer)

브리스톨은 영국에서 문화와 역사, 아름다운 자연을 동시에 즐길 수 있는 매력적인 도시입니다. 3일간의 가성비 좋은 여행 계획을 아래와 같이 제안드립니다.

### 1일차: 브리스톨 시내 탐방

**오전:**
- **브리스톨 대성당(Bristol Cathedral)**: 역사적인 건축물과 아름다운 정원을 무료로 구경하세요.
- **파운틴 스퀘어(Fountain Square)**: 근처의 카페에서 아침 식사를 즐기세요. 스콘이나 영국식 아침 식사를 추천합니다.

**오후:**
- **브리스톨 박물관 및 미술관(Bristol Museum & Art Gallery)**: 다양한 전시를 무료로 관람할 수 있습니다.
- **클리프턴 서스펜션 브리지(Clifton Suspension Bridge)**: 브리지까지 도보로 이동하며 주변 경관을 감상하세요. 다리 위에서 멋진 사진도 찍을 수 있습니다.

**저녁:**
- **하브리스(Harbourside)**: 이 지역의 레스토랑에서 저녁을 즐기세요. 피쉬 앤 칩스나 전통적인 영국 요리를 추천합니다.

### 2일차: 자연과 문화 체험

**오전:**
- **브리스톨 동물원(Bristol Zoo Gardens)**: 동물원은 입장료가 있지만, 다양한 동물을 가까이에서 볼 수 있습니다. 가족 단위 관광에 적합합니다.

**오후:**
- **로프레스트 스튜디오(Lofthouse Studios)**: 지역 아티스트들의 작업실을 방문하고, 다양한 예술 작품을 감상하세요. 일부 스튜디오에서는 무료 투어를 제공합니다.
- **그레이트 브리스톨 스카이워크(Great Bristol Skywalk)**: 도보로 즐길 수 있는 스카이워크에서 도시의 전경을 감상하세요.

**저녁:**
- **로컬 펍 방문**: 브리스톨의 전통 펍에서 로컬 맥주를 즐기며 저녁을 해결하세요.

### 3일차: 근교 탐방 및 쇼핑

**오전:**
- **브리스톨 항구(Bristol Harbour)**: 항구 주변을 산책하며 아침을 즐기세요. 주말에는 마켓이 열리기도 

### [실습] 템플릿을 변경하여 나만의 요리사 챗봇 만들기

- 위의 프롬프트를 아래 주제에 맞게 **변경** 해보기
1. 페르소나: 10년차 셰프
2. 냉장고 속 재료(여러 재료도 가능)를 입력으로 받아 요리명과 레시피 출력
3. `{food}` 에는 여행갈 냉장고 속 재료를 사용자에게 입력받음
4. `answer` 변수를 출력하여 챗봇의 답변 결과 확인 
5. `Langsmith` 에 접속하여 실행 내용 확인

In [37]:
cook_template = """
당신은 10년차 셰프입니다.
냉장고 속의 {food}들을 포함하여 만들 수 있는 요리명과 그에 대한 레시피를 출력해주세요.
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성
prompt = PromptTemplate.from_template(cook_template)

# ChatOpenAI 챗모델을 초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

# 체인을 구성
chain = prompt | model | output_parser

In [38]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"food": "바나나 초콜릿"})
# 스트리밍 출력
stream_response(answer)

물론이죠! 바나나와 초콜릿을 활용한 간단하면서도 맛있는 디저트를 제안합니다. 바로 **초콜릿 바나나 브레드**입니다. 

### 초콜릿 바나나 브레드 레시피

#### 재료
- 잘 익은 바나나 3개
- 설탕 100g (기호에 따라 조절)
- 계란 2개
- 바닐라 추출물 1작은술
- 밀가루 200g
- 베이킹 소다 1작은술
- 소금 1/4작은술
- 초콜릿 칩 100g (기호에 따라 조절)
- 식용유 또는 녹인 버터 80ml

#### 조리 방법

1. **오븐 예열**: 오븐을 175도(섭씨)로 예열합니다.

2. **바나나 으깨기**: 큰 볼에 잘 익은 바나나를 넣고 포크로 으깹니다. 부드러운 퓨레 상태가 되도록 해주세요.

3. **재료 섞기**: 으깬 바나나에 설탕, 계란, 바닐라 추출물, 식용유(또는 녹인 버터)를 넣고 잘 섞어줍니다.

4. **가루 재료 혼합**: 다른 볼에 밀가루, 베이킹 소다, 소금을 섞어준 후, 바나나 혼합물에 천천히 넣고 섞어줍니다. 너무 많이 섞지 않도록 주의하세요.

5. **초콜릿 칩 추가**: 마지막으로 초콜릿 칩을 넣고 가볍게 섞어줍니다.

6. **반죽 붓기**: 반죽을 기름을 바른 9x5인치(23x13cm) 빵 틀에 붓습니다.

7. **굽기**: 예열된 오븐에서 약 60분간 굽습니다. 가운데에 이쑤시개를 찔렀을 때 깨끗하게 나와야 합니다.

8. **식히기**: 빵이 구워지면 오븐에서 꺼내고 틀에서 10분 정도 식힌 후, 틀에서 빼내어 완전히 식힙니다.

9. **서빙**: 식힌 후 슬라이스하여 서빙합니다. 원하신다면 휘핑 크림이나 아이스크림과 함께 즐기세요!

이렇게 간단하게 만들 수 있는 초콜릿 바나나 브레드입니다. 바나나의 자연스러운 단맛과 초콜릿의 풍미가 어우러져 정말 맛있답니다. 즐거운 요리 시간 되세요!

### [실습] 템플릿을 변경하여 나만의 헬스 트레이너 챗봇 만들기

- 위의 프롬프트를 아래 주제에 맞게 **변경** 해보기
1. 페르소나: 10년차 헬스 트레이너
2. 운동하고 싶은 신체 부위를 입력하면 운동 루틴을 출력 
3. `{today}` 에는 운동하고 싶은 신체 부위를 사용자에게 입력받음
4. `answer` 변수를 출력하여 챗봇의 답변 결과 확인 
5. `Langsmith` 에 접속하여 실행 내용 확인

In [41]:
exercise_template = """
당신은 10년차 운동 트레이너입니다.
{today}에 대한 하루동안의 총합 20세트 운동 루틴을 만들어주세요.
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성
prompt = PromptTemplate.from_template(exercise_template)

# ChatOpenAI 챗모델을 초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

# 체인을 구성
chain = prompt | model | output_parser

In [43]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"today": "하체"})
# 스트리밍 출력
stream_response(answer)

하체를 집중적으로 운동할 수 있는 하루 동안의 총합 20세트 루틴을 아래와 같이 구성해 보았습니다. 이 루틴은 다양한 하체 근육군을 골고루 자극할 수 있도록 설계되었습니다.

### 하체 운동 루틴 (총 20세트)

#### 1. 스쿼트 (Squats)
- **세트**: 4세트
- **반복**: 10-12회
- **설명**: 기본 스쿼트를 실시하며, 체중을 발뒤꿈치에 두고 내려가세요.

#### 2. 런지 (Lunges)
- **세트**: 4세트
- **반복**: 10-12회 (각 다리)
- **설명**: 앞쪽으로 한 발을 내딛으며 무릎이 발끝을 넘지 않도록 주의하세요.

#### 3. 레그 프레스 (Leg Press)
- **세트**: 3세트
- **반복**: 10-12회
- **설명**: 레그 프레스 머신을 사용하여 허벅지와 엉덩이를 집중적으로 자극하세요.

#### 4. 데드리프트 (Deadlifts)
- **세트**: 3세트
- **반복**: 8-10회
- **설명**: 바벨 또는 덤벨을 사용하여 하체와 등 근육을 동시에 발달시킵니다.

#### 5. 레그 컬 (Leg Curls)
- **세트**: 3세트
- **반복**: 10-12회
- **설명**: 햄스트링을 강화하기 위해 레그 컬 머신을 이용합니다.

#### 6. 카프 레이즈 (Calf Raises)
- **세트**: 3세트
- **반복**: 12-15회
- **설명**: 종아리 근육을 발달시키기 위해 서서 또는 앉아서 실시합니다.

#### 7. 스텝 업 (Step-Ups)
- **세트**: 2세트
- **반복**: 10-12회 (각 다리)
- **설명**: 박스나 벤치를 이용하여 한 발씩 올라갔다 내려오는 동작입니다.

### 루틴 요약
- 총 세트 수: 20세트
- 전체 시간: 약 60-90분 (세트 간 휴식 포함)

### 추가 팁
- 운동 전 충분한 스트레칭으로 워밍업을 하세요.
- 각 세트 사이에 60-90초의 휴식을 취하는 것이 좋습니다.
- 운동 후에는 쿨다운 및 스트레칭을 통해 